# Heads Up

In this notebook, we start generating random undirected weighted
graphs to not only better approximate real world road network but
also facilitate the feature engineering of nodes and edges

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import mxnet as mx
import logging
from sklearn.metrics import accuracy_score
from utils import plot_g

In [ ]:
NUM_NODE = 100
WEIGHT_MIN = .0
WEIGHT_MAX = 1.

In [ ]:
def generate_low_degree_g(node_size=20, min_out_degree=2, max_out_degree=4, weight_min=WEIGHT_MIN, weight_max=WEIGHT_MAX):
    
    G = nx.Graph()
    G.add_nodes_from(range(0, node_size))
    
    for node in G.nodes:
        
        tmp_nodes = list(G.nodes)
        tmp_nodes.remove(node)
        random.shuffle(tmp_nodes)
        num_of_neighbors = random.randint(min_out_degree, max_out_degree)
#         print(node, out_neighbors)
        
#         G.add_edges_from(map(lambda d:(node, d), out_neighbors))
        
        for tmp_node in tmp_nodes:

            if G.degree(tmp_node) >= max_out_degree \
                or G.degree(node) >= num_of_neighbors:
                # This node has maximum number of neighbors already
                continue
            
            weight = random.uniform(weight_min, weight_max)
            G.add_edge(node, tmp_node, weight=weight)
        
    return G

In [ ]:
G = generate_low_degree_g()
max_degree = max(G.degree, key=lambda d: d[1])[1]
min_degree = min(G.degree, key=lambda d: d[1])[1]
print(max_degree)
print(min_degree)
print(G.number_of_nodes())
print(G.number_of_edges())
plot_g(G)

In [ ]:
def traverse_g_helper(layer_2_layer_dict, G, parent_node, cur_node, step, depth):
    
    if step > depth:
        return
    
    layer_2_layer_dict[step-1][(parent_node, cur_node)] = \
                        filter(lambda edge: edge[1] != parent_node, G.edges(cur_node))
        
    for edge in G.edges(cur_node):
        
        neighbor = edge[1]
        if neighbor == parent_node:
            continue
        
#         print('Step %d: from %d to %d' % (step, cur_node, neighbor))
        traverse_g_helper(layer_2_layer_dict, G, cur_node, neighbor, step+1, depth)
    
def traverse_g(G, src, parent_node=-1, depth=3):
    
    layer_2_layer_dict = [dict() for _ in range(depth)]
    traverse_g_helper(layer_2_layer_dict, G, parent_node, src, 1, depth)
    
    return layer_2_layer_dict

In [ ]:
DST_NEIGHBOR_LAYERS = generate_dst_neighbor_layers(G)
PAIR_NEIGHBOR_LAYERS = generate_pair_neighbor_layers(G)

In [ ]:
DST_NEIGHBOR_LAYERS

In [ ]:
G.get_edge_data(0,2)

In [ ]:
def encode_layers(G, layers, depth=3):
    
    assert type(layers) is list and len(layers) == depth
    ret = [ np.ones(max_out_degree ** (i+1)) for i in range(depth)]
    
    for layer in range(depth):
        
        assert type(layers[layer]) is dict
        
        idx = 0
        
        for _, edge_list in layers[layer].items():
            
            assert type(edge_list) is list
            
            for edge in edge_list:
                ret[layer][idx] = G.get_edge_data(edge[0], edge[1])['weight']
                idx += 1
        
    return reduce(lambda x, y: np.hstack([x, y]), ret)

def encode_sample(G, start, end, dst, depth=3):
    
    dst_layers_encoded = encode_layers(G, DST_NEIGHBOR_LAYERS[dst])
    edge_layers_encoded = encode_layers(G, PAIR_NEIGHBOR_LAYERS[(start, end)])
    return np.hstack([edge_layers_encoded, dst_layers_encoded])

X = []
y = []

for paths in nx.all_pairs_dijkstra_path(G):
    print("src :%d" % paths[0])
    print(paths[1])
    print('')
    src = paths[0]
    for dst, path in paths[1].items():
        
        if dst == src:
            continue
            
        # X = cur_node + dst_node
        # y = next_node (Dijkstra)
        
        cur_node = src
        for mid_node in path[1:]:
            
            print('X:(%d, %d), y:(%d)' % (cur_node, dst, mid_node))
            X.append(encode_sample(G, cur_node, mid_node, dst))
            y.append(0)
            
            cur_node = mid_node
            
            # Retrieve edge(cur_node, mid_node) || dst
            
#         print(cur_node, dst)
        print(path)
        print('')

In [ ]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

In [ ]:
X[0]

In [ ]:
def generate_dst_neighbor_layers(G, depth=3):
    
    ret = dict()
    for node in G.nodes:
        ret[node] = traverse_g(G, node, parent_node=-1, depth=depth)
    
    return ret

def generate_pair_neighbor_layers(G, depth=3):
    
    ret = dict()
    for edge in G.edges:
        ret[(edge[0], edge[1])] = traverse_g(G, edge[1], parent_node=edge[0], depth=depth)
        ret[(edge[1], edge[0])] = traverse_g(G, edge[0], parent_node=edge[1], depth=depth)
    
    return ret

def generate_dataset(G, depth=3):
    
    pass